<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio -qU
!pip install sentence-transformers -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import gradio as gr 
from sentence_transformers import SentenceTransformer,util
import pickle 
import pandas as pd 
from huggingface_hub import hf_hub_download
import os 

In [3]:
pd.options.mode.chained_assignment = None  # Turn off SettingWithCopyWarning

In [5]:
songs_df = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename = 'spotify_transformed.csv'))
mappings = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename ='song_mappings.csv'))
verses_df = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename = 'verses.csv'))
song_embeddings = pickle.load(open(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename ='embeddings.pkl'),'rb'))

In [8]:
mappings.rename(columns={'Unnamed: 0':'verse','0':'song_name'},inplace=True)

In [11]:
verses_df.rename(columns={'0':'verse'},inplace=True)

In [9]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [14]:
def generate_playlist(prompt):
  prompt_embed = embedder.encode(prompt)
  hits = util.semantic_search(prompt_embed,song_embeddings,top_k=30)
  hits = pd.DataFrame.from_dict(hits[0])
  verses_match = verses_df.iloc[hits['corpus_id']]
  songs_match = mappings[mappings['verse'].isin(verses_match['verse'].values)]
  songs_match = songs_df[songs_df['song_name'].isin(songs_match['song_name'].values)]
  songs_match = songs_match.sort_values('song_name')
  songs_match = songs_match.drop_duplicates(subset='song_name')
  songs_name = list(songs_match['song_name'])
  cover_art = list(songs_match['image'])
  images = [gr.Image.update(value=art,visible=True) for art in cover_art]
  return (
      gr.Radio.update(label='songs',interactive=True,choices=songs_name),
      *images
      )
                                

In [15]:
generate_playlist(prompt)

({'choices': ['Culture',
   'Door',
   'Free Mind',
   'Last Last',
   'Logba Logba',
   'Time N Affection',
   'attention'],
  'label': 'songs',
  'show_label': None,
  'interactive': True,
  'visible': None,
  'value': <_Keywords.NO_VALUE: 'NO_VALUE'>,
  '__type__': 'update',
  'mode': 'dynamic'},
 {'label': None,
  'show_label': None,
  'interactive': None,
  'visible': True,
  'value': 'https://i.scdn.co/image/ab67616d0000b27379ab73f6bfebec7c0e85f922',
  '__type__': 'update'},
 {'label': None,
  'show_label': None,
  'interactive': None,
  'visible': True,
  'value': 'https://i.scdn.co/image/ab67616d00001e0218d0121ff57f0b9dfbe84b54',
  '__type__': 'update'},
 {'label': None,
  'show_label': None,
  'interactive': None,
  'visible': True,
  'value': 'https://i.scdn.co/image/ab67616d0000b2730e4f0a913e2ea614cacd8545',
  '__type__': 'update'},
 {'label': None,
  'show_label': None,
  'interactive': None,
  'visible': True,
  'value': 'https://i.scdn.co/image/ab67616d0000b273395d71ee367

In [16]:
def set_example_prompt(examples):
  return gr.TextArea.update(value=examples[0])

In [ ]:
demo = gr.Blocks()
with demo:
  gr.Markdown(
      ''' 
      # A playlist Generator for Afrobeats
      This was built using Sentence Transformers and gradio

      '''
  )
  with gr.Row():
    with gr.Column():
      gr.Markdown(
          '''
          Enter a Prompt to generate a playlist based on that prompt 
          '''
      )
      song_prompt = gr.TextArea(
          value='I need igbo and shayo',
          placeholder = "Enter a sentence that describes how you're feeling or what you want your playlist to be about"
      )
      example_prompts = gr.Dataset(
          components=[song_prompt],
          samples= [
              ['Lazy day'],
              ['I need money'],
              ['nobody gets me ']
          ]
      )
    with gr.Column():
      fetch_songs = gr.Button(value='generate your playlist',).style(full_width=True)
      with gr.Row():
                tile1 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile2 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile3 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
      with gr.Row():
              tile4 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile5 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile6 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
      with gr.Row():
              tile7 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile8 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile9 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)

      # Workaround because of the Gallery issues
      tiles = [tile1, tile2, tile3, tile4, tile5, tile6, tile7, tile8, tile9]

      with gr.Row():
        song_options = gr.Radio(label='songs',interactive=True,choices=None,type='value')
    with gr.Column():
      lyrics = gr.Textbox(label='lyrics',placeholder="select a song to see it's lyrics")

  
    fetch_songs.click(
        fn=generate_playlist,
        inputs=[song_prompt],
        outputs=[song_options,*tiles]
    )
    example_prompts.click(
        fn=set_example_prompt,
        inputs = [example_prompts],
        outputs = example_prompts.components
    )
  

  demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://beb30e8c999b95f42d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
